# Military Drone Strikes, Afgan, Pakistan, Yemen and Somalia

This script is used to prepare our data for future analysis. it reads in the DroneWarsData_tw.xlsx spreadsheet and outputs drone-strikes.xlsx spreadsheet.

We can then use the data in drone-strikes.xlsx in our data analysis piece

The DroneWarsData.xlsx can be downloaded from the dronewars.io website
https://dronewars.github.io/data/DroneWarsData.xlsx

In [ ]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import mitosheet
import re

In [ ]:
# Read in the 'All' sheet
df = pd.read_excel('DroneWarsData.xlsx', sheet_name = 'All')

In [ ]:
# convert our date to a pandas datetime field
df['Date (MM-DD-YYYY)'] = pd.to_datetime(df['Date (MM-DD-YYYY)'])

In [ ]:
# Rename our date column to Date
df.rename(columns = {'Date (MM-DD-YYYY)' : 'Date'}, inplace=True)

In [ ]:
# Add a Year and Month column to our dataframe
df['Year'] = pd.DatetimeIndex(df['Date']).year
df['Month'] =pd.DatetimeIndex(df['Date']).month
df['Day'] = df['Date'].dt.day_name()

## Lets see what we have

In [ ]:
# Check we loaded our data
df.head()

In [ ]:
mitosheet.sheet(df, view_df=True)

In [ ]:
# Lets see what we have
df.info()

In [ ]:
# Check our Column Names
df.columns

In [ ]:
df.info()

## Data Cleansing

### Trim Trailing spaces from columns in Pandas

In [ ]:
# Trim leading and trailing spaces
df['Country'] = df['Country'].str.strip()
df['Most Specific Location'] = df['Most Specific Location'].str.strip()

### Dealing with some of our 'Unknown' values
This code deals with data in our 'Country' column that is set to 'Unknown".
We fix this so when we move on to the next section when we clean up Latitude
and Longitude we have valid Country names.

In [ ]:
# Data Clensing
# Fix Unknown 'Most Specific Location'
# Set our Country Masks
is_yemen = df['Country'] == 'Yemen'
is_somalia = df['Country'] == 'Somalia'
is_pakistan = df['Country'] == 'Pakistan'
is_afghanistan =df['Country'] == 'Afghanistan'

# Set a mask to grab our Unknown 'Most Specific Locations'
# We will use this to set the 'Most Specific Location'
is_unknown = df['Most Specific Location'] == 'Unknown'

# Fix the spelling of Afghanistan
is_afganistan = df['Country'] == 'Afganistan'
df.loc[(is_afganistan), 'Country'] = 'Afghanistan'

# Fix up our Unknown 'Most Specific Locations' by country
df.loc[(is_yemen & is_unknown), 'Most Specific Location'] = 'Yemen, Unknown'
df.loc[(is_somalia & is_unknown), 'Most Specific Location'] = 'Somalia, Unknown'
df.loc[(is_pakistan & is_unknown), 'Most Specific Location'] = 'Pakistan, Unknown'
df.loc[(is_afghanistan & is_unknown), 'Most Specific Location'] = 'Afghanistan, Unknown'

### Setting Lat and Long for our previously 'Unknown' locations
Here I set the Lat and Long for any 'Unknown' detected in the step above. I have used a fefault Lat/Long pair for each Country. This was just chosen at random from Google Maps

In [ ]:
# Get a list off all the rows where there is an unknown for "Most Specific Lat/Long"
ms_latlong = df["Most Specific Lat/Long"] == 'Unknown'

# Using our Country mask from the step above and our new ms_latlong mask we can assign
# default lat long co-ords to any data in the'Most Specific Lat/Long' column
df.loc[(is_yemen & ms_latlong), "Most Specific Lat/Long"] = '14.454376, 45.102241'
df.loc[(is_somalia & ms_latlong), "Most Specific Lat/Long"] = '-1.496491, 41.592160'
df.loc[(is_pakistan & ms_latlong), "Most Specific Lat/Long"] = '33.976996, 69.951233'
df.loc[(is_afghanistan & ms_latlong), "Most Specific Lat/Long"] = '29.484804, 62.761735'

### Dealing with a particular Excel error in our dataset
Because the dataset has used data from calculated fields in our initial spreadsheet,  some of these claculated fields were not valid and Excel marks them as '#VALUE!' error. When we import this data into pandas the errors come across.

In [ ]:
# Fix '#VALUE!'
is_value = df['Other (Non-civilian/children Killed)'] == '#VALUE!'
df.loc[(is_value), 'Other (Non-civilian/children Killed)'] = 0

### Replacing one data value for another in specific columns

In [ ]:
# Replace the minus sign in the column list below
col = ['Minimum total people killed', 'Maximum total people killed',
       'Minimum civilians reported killed','Maximum civilians reported killed',
       'Minimum children reported killed', 'Maximum children reported killed',
       'Minimum reported injured' ]

for i in col:
  df[i] = df[i].replace('-', "0")

### Fix the erroneous data in the 'Most Specific Lat/Long'
Because humans added this data somewhere they have also intriduced errors. 
Using a regular expresion we can clean up our data in this column efficently

In [ ]:
# Create list to hold our lLatitude and Longitude cleaned data
lat_list = []
long_list = []

# Set the Regular Expresion that we want to find in our data
pattern = re.compile(r"-?\d\d?\.\d+")

# Loop through our column and extract the Lat and long values and append to our lists
for pair in df['Most Specific Lat/Long']:
    match = pattern.findall(pair)
    if len(match) == 2:
        lat, long = match
        lat_list.append(lat)
        long_list.append(long)

# Set the 'Latitude' and 'Longitude' values to our cleaned values
df['Latitude'] = lat_list
df['Longitude'] = long_list

In [ ]:
mitosheet.sheet(df, view_df=True)

### Set our column datatypes

In [ ]:
# Sort out our data types
df = df.astype({'Latitude': 'float64',
                'Longitude' : 'float64',
                'Minimum total people killed': 'int64',
                'Maximum total people killed': 'int64',
                'Minimum civilians reported killed': 'int64',
                'Maximum civilians reported killed': 'int64',
                'Minimum children reported killed': 'int64',
                'Maximum children reported killed': 'int64',
                'Minimum reported injured': 'int64',
                'Other (Non-civilian/children Killed)': 'int64'
               })

In [ ]:
df.info()

### Save our cleansed data to a new spreadsheet

In [ ]:
# Export our newly formated dataframe to excel but drop our pandas index column
df.to_excel('C:/Users/twelsh/data/Notebooks/drone-strikes/published/drone-strikes.xlsx', index=False)